# Projekt: Prisprognos för Begagnade Bilar 🚗💰

**Mål:** Vi ska bygga en Machine Learning-modell som kan gissa priset på en bil baserat på dess egenskaper.

**Metod:**
1. **Data Cleaning:** Göra om text till siffror (One-Hot Encoding).
2. **Feature Engineering:** Skapa smartare variabler (t.ex. "Bilens ålder" istället för "Årtal").
3. **Model Battle:** Vi ställer 4 olika algoritmer mot varandra för att se vem som har lägst felmarginal (RMSE).

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Importera modellerna vi ska tävla med
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Läs in datan (semikolon-separerad)
# Vi använder pandas för att skapa en DataFrame (en tabell)
df = pd.read_csv('car_price_dataset.csv', sep=';')

# Visa de första 5 raderna för att se vad vi jobbar med
print("--- Rådata (De första raderna) ---")
display(df.head())

--- Rådata (De första raderna) ---


,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867


## Steg 1: Datastädning (One-Hot Encoding)
Datorer kan inte räkna med ord som "Volvo" eller "Diesel". De behöver siffror.

Vi använder **One-Hot Encoding** för att göra om kategorier till siffror:
* En kolumn `Fuel_Type` med värdet "Diesel" blir...
* En ny kolumn `Fuel_Type_Diesel` med värdet **1** (för ja) eller **0** (för nej).

In [69]:
# 'drop_first=True' tar bort den första kategorin för att undvika dubbelinformation.
# (Om det t.ex. inte är Bensin eller Diesel, så vet modellen automatiskt att det är El/Hybrid).
df_encoded = pd.get_dummies(df, drop_first=True)

print(f"Antal kolumner efter städning: {df_encoded.shape[1]}")
print(df_encoded.head())
display(df_encoded.head())

Antal kolumner efter städning: 49
   Year  Engine_Size  Mileage  Doors  Owner_Count  Price  Brand_BMW  \
0  2020          4.2   289944      3            5   8501      False   
1  2012          2.0     5356      2            3  12092      False   
2  2020          4.2   231440      4            2  11171      False   
3  2023          2.0   160971      2            1  11780      False   
4  2003          2.6   286618      3            3   2867      False   

   Brand_Chevrolet  Brand_Ford  Brand_Honda  ...  Model_Sonata  \
0            False       False        False  ...         False   
1             True       False        False  ...         False   
2            False       False        False  ...         False   
3            False       False        False  ...         False   
4            False       False        False  ...         False   

   Model_Sportage  Model_Tiguan  Model_Tucson  Model_X5  Fuel_Type_Electric  \
0           False         False         False     False        

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price,Brand_BMW,Brand_Chevrolet,Brand_Ford,Brand_Honda,...,Model_Sonata,Model_Sportage,Model_Tiguan,Model_Tucson,Model_X5,Fuel_Type_Electric,Fuel_Type_Hybrid,Fuel_Type_Petrol,Transmission_Manual,Transmission_Semi-Automatic
0,2020,4.2,289944,3,5,8501,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,2012,2.0,5356,2,3,12092,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
2,2020,4.2,231440,4,2,11171,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2023,2.0,160971,2,1,11780,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
4,2003,2.6,286618,3,3,2867,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True


## Steg 2: Feature Engineering (Göra datan smartare) 🧠
Här använder vi vår mänskliga logik för att hjälpa modellen.

1. **Ålder istället för Årtal:** En modell förstår "8 år gammal" bättre än "Tillverkad 2016".
2. **Mil per År (Intensitet):** En bil som gått 20 000 mil på 2 år är mer sliten än en som gått samma sträcka på 10 år.

Vi skapar dessa variabler nu.

In [70]:
# Vi jobbar på en kopia för att inte förstöra originalet
df_fe = df_encoded.copy()

# 1. Skapa "Car_Age" (Vi antar att året är 2024)
df_fe['Car_Age'] = 2024 - df['Year']

# Fix: Om bilen är helt ny (0 år), sätt ålder till 1 för att undvika division med noll
df_fe['Car_Age'] = df_fe['Car_Age'].replace(0, 1)

# 2. Skapa "Miles_Per_Year" (Hur hårt bilen körts)
df_fe['Miles_Per_Year'] = df_fe['Mileage'] / df_fe['Car_Age']

# 3. Städa bort 'Year'
# Nu när vi har 'Age' behöver vi inte 'Year'. Det blir bara brus för modellen.
df_fe = df_fe.drop(['Year'], axis=1)

# Visa de nya super-kolumnerna
print("--- Nya Smartare Features ---")
display(df_fe[['Car_Age', 'Miles_Per_Year', 'Price']].head())

--- Nya Smartare Features ---


,Car_Age,Miles_Per_Year,Price
0,4,72486.000000,8501
1,12,446.333333,12092
2,4,57860.000000,11171
3,1,160971.000000,11780
4,21,13648.476190,2867


## Steg 3: Dela upp i Träning och Test
Vi måste gömma en del av datan (20%) för att kunna testa modellen ärligt.
* **X:** Alla ledtrådar (Miltal, Ålder, Märke...)
* **y:** Facit (Priset)

In [71]:
## Steg 4: The Model Battle Royale 🥊
Nu ställer vi fyra olika "hjärnor" mot varandra:

1. **Linear Regression:** Den enkla standardmodellen. Drar räta linjer.
2. **Random Forest:** En "skog" av beslutsträd. Bra på krångliga mönster.
3. **Gradient Boosting:** Bygger träd stegvis för att rätta sina egna fel. Ofta vinnare i tävlingar.
4. **Smart Polynom (x²):** En linjär modell som får se variablerna "upphöjt till 2".

Vi mäter felet i **RMSE** (Root Mean Squared Error). Det betyder ungefär "Genomsnittligt fel i kronor".
**Lägre är bättre!**

SyntaxError: invalid character '²' (U+00B2) (2987236679.py, line 7)